# Annotate relative radial distance from nuclear center

For each read in the PGP1 dataset, find its normalized radial distance from the nuclear center.

In [1]:
import importlib
import os, sys, warnings
import numpy as np
import pandas as pd

sys.path.append(os.path.abspath(os.path.join(r"..", r".")))

from source import const
from source import radial as rd

In [2]:
pgp_file = "../data/Table_S1_pgp1_data_table.csv"
pgp_data = pd.read_csv(pgp_file,index_col=False)

#Retreive the single cells
cells = []
cell_indices = pgp_data["cell_id"].unique()
for cell_index in cell_indices:
    cells.append(pgp_data.loc[pgp_data["cell_id"] == cell_index])

In [3]:
KEYS = const.get_genome_keys('hg38')

dim = 3

#center cells at origin
centered_cells = []
for cell in cells:
    hull = rd.get_hull(cell, dim=dim)
    origin = rd.get_hull_center(hull, dim=dim)
    centered_cell = rd.center_cell(cell, origin, dim=dim)
    centered_cells.append(centered_cell)

annotated_data = pgp_data.copy() #annotate this

dims = [2,3] #get relative radial distance in 2 and 3-D

for dim in dims:
    for cell in centered_cells:
        hull = rd.get_hull(cell,dim=dim)

        #find relative radial dist for each point in cell
        for index, row in cell.iterrows(): 

            #Get spatial position vectors for each read
            R = []
            for i in range(dim):
                R.append(row[KEYS['dim'][i]])
            R = np.array(R)

            r_rel = rd.get_r_rel(hull, R) 
            key = 'norm_r_' +str(dim) + "D"
            annotated_data.at[index, key] = r_rel

annotated_data.to_csv("../data/Table_S1_pgp1_data_table.csv",index=False)